# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment


The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
from unityagents import UnityEnvironment
import numpy as np
import pandas as pd
import torch
from collections import namedtuple, deque

env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:


for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    for i in itertools.count():
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                 # exit loop if episode finished
            print('ended after %s iterations' % i)
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 4. Train

In [8]:
from ddpg_agent import Agent 

left_agent = Agent(state_size=state_size, action_size=action_size, random_seed=10)
right_agent = left_agent # Agent(state_size=state_size, action_size=action_size, random_seed=10)

def ddpg(n_episodes=20000, max_t=3000):
    left_scores_deque = deque(maxlen=100)
    left_scores = []
    right_scores_deque = deque(maxlen=100)
    right_scores = []
    max_scores_deque = deque(maxlen=100)
    max_scores = []

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] 
        states = env_info.vector_observations
        left_agent.reset()
        right_agent.reset()
        left_score = 0
        right_score = 0
        for i in range(1, max_t):
            
            actions = np.concatenate([left_agent.act(states[0:1]), right_agent.act(states[1:2])])
                
            actions = np.clip(actions, -1, 1) 
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                        # get reward (for each agent)
            joint_reward = np.sum(rewards) 
            dones = env_info.local_done                        # see if episode finished

            left_score += rewards[0]
            right_score+= rewards[1]
            
            
            left_agent.step( states[0], actions[0], rewards[0], next_states[0], dones[0])
            right_agent.step(states[1], actions[1], rewards[1], next_states[1], dones[1])

            states = next_states
            
            if np.any(dones):
                break

        left_scores_deque.append(left_score)
        left_scores.append(left_score)
        right_scores_deque.append(right_score)
        right_scores.append(right_score)
        max_score = max(left_score, right_score)
        max_scores_deque.append(max_score)
        max_scores.append(max_score)
        print('\rEpisode {}\tAvg Left Score: {:.3f}\tAvg Right Score: {:.3f}\tAvg Max Score: {:.3f}\tLast Score: {:.3f}'
             .format(i_episode, np.mean(left_scores_deque), np.mean(right_scores_deque), np.mean(max_scores_deque), max_score ), end="")
        if i_episode % 100 == 0:
            # print("exploration scale: %s" % left_agent.exploration_scale)
            torch.save(left_agent.actor_local.state_dict(), 'left_checkpoint_actor.pth')
            torch.save(left_agent.critic_local.state_dict(), 'left_checkpoint_critic.pth')
            torch.save(right_agent.actor_local.state_dict(), 'right_checkpoint_actor.pth')
            torch.save(right_agent.critic_local.state_dict(), 'right_checkpoint_critic.pth')
            print('\rEpisode {}\tAvg Left Score: {:.3f}\tAvg Right Score: {:.3f}\tAvg Max Score: {:.3f}'
                   .format(i_episode, np.mean(left_scores_deque), np.mean(right_scores_deque), np.mean(max_scores_deque)))
    return scores

scores = ddpg()

Episode 4	Avg Left Score: -0.002	Avg Right Score: -0.007	Avg Max Score: 0.000	Last Score: 0.000

C:\Users\z00110mk.AD001\git\deep-reinforcement-learning-project-3\ddpg_agent.py:114: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), GRAD_CLIP)


Episode 100	Avg Left Score: 0.000	Avg Right Score: -0.002	Avg Max Score: 0.006	Last Score: 0.100exploration scale: 0.5670636698003494
Episode 100	Avg Left Score: 0.000	Avg Right Score: -0.002	Avg Max Score: 0.006
Episode 200	Avg Left Score: 0.004	Avg Right Score: -0.002	Avg Max Score: 0.012	Last Score: 0.000exploration scale: 0.3083288433677974
Episode 200	Avg Left Score: 0.004	Avg Right Score: -0.002	Avg Max Score: 0.012
Episode 300	Avg Left Score: 0.033	Avg Right Score: 0.030	Avg Max Score: 0.053	Last Score: 0.100exploration scale: 0.10665891501555477
Episode 300	Avg Left Score: 0.033	Avg Right Score: 0.030	Avg Max Score: 0.053
Episode 400	Avg Left Score: 0.014	Avg Right Score: 0.010	Avg Max Score: 0.023	Last Score: 0.100exploration scale: 0.050464079352414384
Episode 400	Avg Left Score: 0.014	Avg Right Score: 0.010	Avg Max Score: 0.023
Episode 500	Avg Left Score: 0.029	Avg Right Score: 0.020	Avg Max Score: 0.044	Last Score: 0.100exploration scale: 0.018444356586939055
Episode 500	Av

KeyboardInterrupt: 

When finished, you can close the environment.

In [ ]:
env.close()